In [1]:
# Application control functions

import os
import webbrowser
import subprocess

def open_chrome():
    """
    Opens Google Chrome browser.
    
    Usage: Launch Chrome, Open Chrome browser, Start Chrome
    """
    webbrowser.open("https://www.google.com")

def open_firefox():
    """
    Opens Firefox browser.
    
    Usage: Launch Firefox, Open Firefox browser, Start Firefox
    """
    webbrowser.open("https://www.mozilla.org")

def open_calculator():
    """
    Opens the calculator application.
    
    Usage: Open calculator, Launch calc, Start calculator app
    """
    if os.name == 'nt':  # Windows
        os.system("calc")
    elif os.name == 'posix':  # macOS or Linux
        if os.uname().sysname == 'Darwin':  # macOS
            os.system("open -a Calculator")
        else:  # Linux
            os.system("gnome-calculator")

def open_notepad():
    """
    Opens the notepad/text editor application.
    
    Usage: Open notepad, Launch text editor, Start notepad
    """
    if os.name == 'nt':  # Windows
        os.system("notepad")
    elif os.name == 'posix':  # macOS or Linux
        if os.uname().sysname == 'Darwin':  # macOS
            os.system("open -a TextEdit")
        else:  # Linux
            os.system("gedit")

def open_file_explorer():
    """
    Opens the file explorer/finder.
    
    Usage: Open file explorer, Launch finder, Open file manager
    """
    if os.name == 'nt':  # Windows
        os.system("explorer")
    elif os.name == 'posix':  # macOS or Linux
        if os.uname().sysname == 'Darwin':  # macOS
            os.system("open .")
        else:  # Linux
            os.system("xdg-open .")

In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer

class EmbeddingModel:
    def __init__(self, model="Alibaba-NLP/gte-large-en-v1.5"):
        """
        Initializes the embedding model.
        
        Args:
            model_name (str): The name of the Hugging Face Embedding model to use
        """
        self.embeddings = SentenceTransformer(model, trust_remote_code=True)
    
    def create_embedding(self, text: str):
        """
        Creates an embedding for the given text.
        
        Args:
            text (str): The text to create an embedding for
            
        Returns:
            numpy.ndarray: The embedding vector
        """
        return self.embeddings.encode(text)

In [3]:
import sys
import os

project_root = r"C:\Users\Swapnil Singh\Desktop\LLM+RAG API\function_execution_api"
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
import os
import inspect
import importlib

def collect_function_metadata():
    folder_path = os.path.join(project_root, "app", "function_registry")
    all_metadata = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".py") and filename != "__init__.py":
            module_name = filename[:-3]  # Remove '.py' extension
            module_path = os.path.join(folder_path, filename)

            # Load module dynamically
            spec = importlib.util.spec_from_file_location(module_name, module_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)

            for name, obj in inspect.getmembers(module, inspect.isfunction):
                docstring = inspect.getdoc(obj) or "No docstring available"

                all_metadata.append({
                    "module": module_name,
                    "function_name": name,
                    "description": docstring,
                })

    return all_metadata

In [5]:
metadata = collect_function_metadata()
metadata

[{'module': 'application_functions',
  'function_name': 'open_calculator',
  'description': 'Opens the calculator application.\n\nUsage: Open calculator, Launch calc, Start calculator app'},
 {'module': 'application_functions',
  'function_name': 'open_chrome',
  'description': 'Opens Google Chrome browser.\n\nUsage: Launch Chrome, Open Chrome browser, Start Chrome'},
 {'module': 'application_functions',
  'function_name': 'open_file_explorer',
  'description': 'Opens the file explorer/finder.\n\nUsage: Open file explorer, Launch finder, Open file manager'},
 {'module': 'application_functions',
  'function_name': 'open_firefox',
  'description': 'Opens Firefox browser.\n\nUsage: Launch Firefox, Open Firefox browser, Start Firefox'},
 {'module': 'application_functions',
  'function_name': 'open_notepad',
  'description': 'Opens the notepad/text editor application.\n\nUsage: Open notepad, Launch text editor, Start notepad'},
 {'module': 'system_functions',
  'function_name': 'execute_she

In [5]:
from pymilvus import connections, db, utility, FieldSchema, DataType, Collection, CollectionSchema

In [7]:
conn = connections.connect("default", host="localhost", port="19530")

try:
    # Checking if the "default" connection is active
    if connections.has_connection("default"):
        print("Connection to Milvus is successful!")

except Exception as e:
    print("Error while checking Milvus connection:", e)

Connection to Milvus is successful!


In [8]:
db.list_database()

['default', 'metadb']

In [9]:
db.using_database(db_name="metadb")

In [10]:
utility.list_collections()

['functions']

In [16]:
# SChema
id_field = FieldSchema(name = "id", dtype=DataType.INT64,is_primary=True, auto_id = True)
Module_field = FieldSchema(name = "Module", dtype=DataType.VARCHAR, max_length = 255)
funcName_field = FieldSchema(name = "Function_Name", dtype=DataType.VARCHAR, max_length = 255)
description_field = FieldSchema(name = "Metadata", dtype=DataType.FLOAT_VECTOR, dim=1024)
sourceCode_field = FieldSchema(name = "Source_Code", dtype=DataType.VARCHAR, max_length = 2048)

schema = CollectionSchema(fields=[id_field, Module_field, funcName_field, description_field, sourceCode_field])

In [10]:
collection = Collection(name='functions')

In [13]:
utility.list_collections()

['functions']

In [19]:
index_params = {
    "index_type": "HNSW",
    "metric_type": "L2",
    "params": {
        "M": 20,
        "efConstruction": 300
    }
}

In [20]:
collection.create_index(field_name="Metadata", index_params=index_params)

Status(code=0, message=)

In [11]:
collection.load()

In [40]:
# Data Ingestion
i = 0
for data in metadata:
    print(f"{i} : {data['description']} : {type(data['description'])}")
    i = i + 1

0 : Opens the calculator application.

Usage: Open calculator, Launch calc, Start calculator app : <class 'str'>
1 : Opens Google Chrome browser.

Usage: Launch Chrome, Open Chrome browser, Start Chrome : <class 'str'>
2 : Opens the file explorer/finder.

Usage: Open file explorer, Launch finder, Open file manager : <class 'str'>
3 : Opens Firefox browser.

Usage: Launch Firefox, Open Firefox browser, Start Firefox : <class 'str'>
4 : Opens the notepad/text editor application.

Usage: Open notepad, Launch text editor, Start notepad : <class 'str'>
5 : Executes a shell command and returns the output.

Usage: Run command, Execute shell, Run terminal command

Args:
    command (str): The shell command to execute : <class 'str'>
6 : Retrieves current CPU usage percentage.

Usage: Get CPU usage, Check processor load, Monitor CPU : <class 'str'>
7 : Retrieves disk usage information.

Usage: Get disk space, Check storage, Monitor disk usage : <class 'str'>
8 : Retrieves current RAM usage info

In [24]:
module = [data['module'] for data in metadata]
module

['application_functions',
 'application_functions',
 'application_functions',
 'application_functions',
 'application_functions',
 'system_functions',
 'system_functions',
 'system_functions',
 'system_functions',
 'system_functions',
 'utility_functions',
 'utility_functions',
 'utility_functions']

In [25]:
function_name = [data['function_name'] for data in metadata]
source_code = [data['source_code'] for data in metadata]

In [12]:
embedding = EmbeddingModel()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\Swapnil Singh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Swapnil Singh\.cache\huggingface\hub\models--Alibaba-NLP--gte-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [46]:
description = [embedding.create_embedding(data['description']) for data in metadata]

In [48]:
len(description)

13

In [51]:
len(function_name)

13

In [52]:
collection.insert(data=[module, function_name, description, source_code])

(insert count: 13, delete count: 0, upsert count: 0, timestamp: 456984264580005890, success count: 13, err count: 0

In [55]:
collection.load()

In [15]:
embedding.create_embedding("What is there?")

array([ 0.12094295, -0.97203547, -0.66849756, ...,  0.91345763,
       -1.3141116 , -0.02443987], shape=(1024,), dtype=float32)

In [13]:
def search_milvus(query, collection, limit=1):
    query_embedding = embedding.create_embedding(query)

    search_params = {
    "index_type": "HNSW",
    "metric_type": "L2",
    "params": {
        "M": 20,
        "efConstruction": 300
    }
    }

    results = collection.search(
        data = [query_embedding],
        anns_field = "Metadata",
        param = search_params,
        limit = limit,
        output_fields = ["Module", "Function_Name", "Source_Code"]
    )

    return results

In [14]:
results = search_milvus("Open File explorer", collection)

In [15]:
results

data: ['[\'id: 456977561872627163, distance: 194.1004638671875, entity: {\\\'Module\\\': \\\'application_functions\\\', \\\'Function_Name\\\': \\\'open_file_explorer\\\', \\\'Source_Code\\\': \\\'def open_file_explorer():\\\\n    """\\\\n    Opens the file explorer/finder.\\\\n    \\\\n    Usage: Open file explorer, Launch finder, Open file manager\\\\n    """\\\\n    if os.name == \\\\\\\'nt\\\\\\\':  # Windows\\\\n        os.system("explorer")\\\\n    elif os.name == \\\\\\\'posix\\\\\\\':  # macOS or Linux\\\\n        if os.uname().sysname == \\\\\\\'Darwin\\\\\\\':  # macOS\\\\n            os.system("open .")\\\\n        else:  # Linux\\\\n            os.system("xdg-open .")\\\\n\\\'}\']']

In [19]:
for result in results[0]:
    print(result)

id: 456977561872627164, distance: 304.92315673828125, entity: {'Module': 'application_functions', 'Function_Name': 'open_firefox', 'Source_Code': 'def open_firefox():\n    """\n    Opens Firefox browser.\n    \n    Usage: Launch Firefox, Open Firefox browser, Start Firefox\n    """\n    webbrowser.open("https://www.mozilla.org")\n'}


In [16]:
def filter_results(results):
    data = {}

    module = []
    name = []
    code = []
    
    for result in results[0]:
        name.append(result.entity.get("Function_Name"))
        module.append(result.entity.get("Module"))
        code.append(result.entity.get("Source_Code"))

    data["Module"] = module
    data["Function Name"] = name
    data["Source Code"] = code

    return data

In [17]:
output = filter_results(results)

In [18]:
output

{'Module': ['application_functions'],
 'Function Name': ['open_file_explorer'],
 'Source Code': ['def open_file_explorer():\n    """\n    Opens the file explorer/finder.\n    \n    Usage: Open file explorer, Launch finder, Open file manager\n    """\n    if os.name == \'nt\':  # Windows\n        os.system("explorer")\n    elif os.name == \'posix\':  # macOS or Linux\n        if os.uname().sysname == \'Darwin\':  # macOS\n            os.system("open .")\n        else:  # Linux\n            os.system("xdg-open .")\n']}

In [41]:
output.get('Source Code')[0]

'def open_firefox():\n    """\n    Opens Firefox browser.\n    \n    Usage: Launch Firefox, Open Firefox browser, Start Firefox\n    """\n    webbrowser.open("https://www.mozilla.org")\n'

In [23]:
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [20]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
    model="qwen/qwen-2.5-coder-32b-instruct:free"
)

In [21]:
result = llm.invoke("What is the capital of India?")
print(result.content)

The capital of India is New Delhi.


In [22]:
template = """ Generate a structured Python script with the following constraints:
Function Details:
- Module Name: {Module}
- Function Name: {Function Name}

### **Code Requirements**
1. Import the function from the specified module {Module}.
2. Use a `main()` function to execute the function.
3. Implement error handling with `try-except`.
4. Print a success message if execution is successful.
5. Ensure the script is executable as a standalone module.
6. No additional text, explanations, or markdown formatting (dont use --> ```python `, etc.).

Now, generate only the clean Python script:
"""

In [23]:
template = PromptTemplate(template = template, input_variables = ['Module', 'Function Name'])

In [24]:
prompt = template.invoke({
    "Module" : output.get('Module')[0],
    'Function Name' : output.get('Function Name')[0]
})

In [25]:
clean_code = llm.invoke(prompt)

In [26]:
print(clean_code.content)

from application_functions import open_file_explorer

def main():
    try:
        open_file_explorer()
        print("File explorer opened successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


In [28]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pymilvus import Collection, connections
#from utility_functions import get_current_time  # Example function import
#from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
import os

# Initialize FastAPI app
app = FastAPI()

# Connect to Milvus
conn = connections.connect("default", host="localhost", port="19530")
db.using_database(db_name="metadb")
collection = Collection(name='functions')
collection.load()

# OPENAi LLM setup
#api_token = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= "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",
    model="qwen/qwen-2.5-coder-32b-instruct:free"
)

# Request schema
class ExecuteRequest(BaseModel):
    prompt: str

# def search_milvus(query, collection, limit=1):
#     """Search for a function in Milvus based on user prompt"""
#     from sentence_transformers import SentenceTransformer

#     embedding_model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
#     query_embedding = embedding_model.encode(query)

#     search_params = {
#         "index_type": "HNSW", 
#         "metric_type": "L2", 
#         "params": {
#             "M": 20, 
#             "efConstruction": 300}
#     }

#     results = collection.search(
#         data=[query_embedding], anns_field="Metadata", param=search_params, limit=limit,
#         output_fields=["Module", "Function_Name"]
#     )

#     if not results or len(results[0]) == 0:
#         return None

#     best_match = results[0][0].entity
#     return {
#         "module": best_match.get("Module"),
#         "function_name": best_match.get("Function_Name"),
#     }

def search_milvus(query, collection, limit=1):
    query_embedding = embedding.create_embedding(query)

    search_params = {
    "index_type": "HNSW",
    "metric_type": "L2",
    "params": {
        "M": 20,
        "efConstruction": 300
    }
    }

    results = collection.search(
        data = [query_embedding],
        anns_field = "Metadata",
        param = search_params,
        limit = limit,
        output_fields = ["Module", "Function_Name", "Source_Code"]
    )

    return results

def filter_results(results):
    data = {}

    module = []
    name = []
    code = []
    
    for result in results[0]:
        name.append(result.entity.get("Function_Name"))
        module.append(result.entity.get("Module"))
        code.append(result.entity.get("Source_Code"))

    data["Module"] = module
    data["Function Name"] = name

    return data

def generate_code(output):
    """Generate Python script for the retrieved function"""
    
    template = """ Generate a structured Python script with the following constraints:
    Function Details:
    - Module Name: {Module}
    - Function Name: {Function Name}
    
    ### **Code Requirements**
    1. Import the function from the specified module {Module}.
    2. Use a `main()` function to execute the function.
    3. Implement error handling with `try-except`.
    4. Print a success message if execution is successful.
    5. Ensure the script is executable as a standalone module.
    6. No additional text, explanations, or markdown formatting (dont use --> ```python `, etc.).
    
    Now, generate only the clean Python script:
    """
    
    template = PromptTemplate(template = template, input_variables = ['Module', 'Function Name'])

    prompt = template.invoke({
    "Module" : output.get('Module')[0],
    'Function Name' : output.get('Function Name')[0]
    })
    
    return output.get('Function Name')[0], model.invoke(prompt)

@app.post("/execute")
def execute_function(request: ExecuteRequest):
    """API endpoint to retrieve function based on user prompt"""
    result = search_milvus(request.prompt, collection)
    filter_data = filter_results(result) 
    
    if not result:
        raise HTTPException(status_code=404, detail="No matching function found")

    func_name, generated_code = generate_code(filter_data)

    return {"function": func_name, "code": generated_code.content}

In [29]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI

# Allow Jupyter Notebook to run async loops
nest_asyncio.apply()

In [30]:
# Run FastAPI inside Jupyter
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [25564]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57340 - "POST /execute HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25564]
